# Part 1 - Visualise the provided calibration data

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys
import os


# add SRC to path to allow imports
sys.path.append(os.path.join("..", "src"))

from make_dataset import make_dataset_from_raw

In [ ]:
path = os.path.join("..", "data", "raw", "calibration.csv")
df, info_calibr = make_dataset_from_raw(path)

## Plotting

Plot the measured absorbance (y-axis) 
against the wavelengths (x-axis) 
for the different dilutions

Plot both raw and corrected
3 corrected readings - corrected is minus the blank 



In [ ]:
# colour map from px.colors.qualitative.D3
colours = [
    "#1F77B4",
    "#FF7F0E",
    "#2CA02C",
    "#D62728",
    "#9467BD",
    "#8C564B",
    "#E377C2",
    "#7F7F7F",
    "#BCBD22",
    "#17BECF",
]

In [ ]:
fig = go.Figure()

# loop though each dilution
dilutions = df["dilution_sample"].unique()
for i, dilution in enumerate(dilutions):
    df_plot = df.query("dilution_sample == @dilution")
    # add main line
    fig.add_trace(
        go.Scatter(
            x=df_plot["wavelength_nm"],
            y=df_plot["mean_absorption_sample"],
            mode="lines",
            name=f"Dilution: {dilution}",
            line=dict(color=colours[i]),
        )
    )
    # remove as too cluttered without more formatting
    #   TODO: make ub.lb transparent, remove from legend
    # # add upper/lower
    # fig.add_trace(
    #     go.Scatter(
    #         x=df_plot["wavelength_nm"],
    #         y=(
    #             df_plot["mean_absorption_sample"] +
    #                 df_plot["absorption_std_sample"]
    #            ),
    #         mode="lines",
    #         # name=f"Dilution: {dilution} upper",
    #         line=dict(color=colours[i]),
    #         )
    #         )
    # fig.add_trace(
    #     go.Scatter(
    #         x=df_plot["wavelength_nm"],
    #         y=(
    #             df_plot["mean_absorption_sample"] -
    #                 df_plot["absorption_std_sample"]
    #            ),
    #         mode="lines",
    #         # name=f"Dilution: {dilution} lower",
    #         line=dict(color=colours[i]),
    #         )
    #         )


fig.update_layout(
    title=f'Measured Absorbance Spectra from test ID {file_info["test_id"]}',
    xaxis_title="Wavelength (nm)",
    yaxis_title="Absorption (OD)",
    template="plotly_white",
)

fig.show()

In [ ]:
fig = go.Figure()

# loop though each dilution
dilutions = df["dilution_sample"].unique()
for i, dilution in enumerate(dilutions):
    df_plot = df.query("dilution_sample == @dilution")
    # add main line
    fig.add_trace(
        go.Scatter(
            x=df_plot["wavelength_nm"],
            y=df_plot["corrected_mean_absorption_sample"],
            mode="lines",
            name=f"Dilution: {dilution}",
            line=dict(color=colours[i]),
        )
    )

fig.update_layout(
    title=f'Corrected Absorbance Spectra from test ID {file_info["test_id"]}',
    xaxis_title="Wavelength (nm)",
    yaxis_title="Absorption (OD)",
    template="plotly_white",
)

fig.show()

TODO:
* Filter LHS
* Correct colour scale to be sequential 
* Add in upper and lower lines, fill between maybe?
* Add more descriptive text to plot, e.g. subtitle, dates, annotations, etc. 
* Deal with outliers in data